In [73]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

from keras.applications.vgg16 import VGG16 # load VGG16 model from keras
from keras.preprocessing.image import load_img # Using images library
from keras.preprocessing.image import img_to_array # # convert image pixels to numpy for specific manipulations
from keras.applications.vgg16 import preprocess_input # to prepare for new input
from keras.applications.vgg16 import decode_predictions # for reporting probabilities

In [74]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# prepare an iterators for each dataset
train_it = train_datagen.flow_from_directory('data/images/train',
                                     target_size=(224,224),
                                     color_mode='rgb',
                                     batch_size=32,
                                     class_mode='categorical',
                                     shuffle=True)
# val_it = datagen.flow_from_directory('data/validation/', class_mode='categorical')
test_it = train_datagen.flow_from_directory('data/images/test',
                                     target_size=(224,224),
                                     color_mode='rgb',
                                     batch_size=32,
                                     class_mode='categorical',
                                     shuffle=True)
# confirm the iterator works
train_batchX, train_batchy = train_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (train_batchX.shape, train_batchX.min(), train_batchX.max()))
test_batchX, test_batchy = test_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (test_batchX.shape, test_batchX.min(), test_batchX.max()))

Found 220 images belonging to 5 classes.
Found 16 images belonging to 5 classes.
Batch shape=(32, 224, 224, 3), min=-123.680, max=151.061
Batch shape=(16, 224, 224, 3), min=-123.680, max=151.061


In [75]:
# classifying only weapons 
vgg16_model = VGG16(weights="imagenet")

In [76]:
from keras.models import Sequential
from keras.layers import Dense, Activation

seq_model = Sequential()

# iterate all layers in vgg16 to sequential for custom modification
for layer in vgg16_model.layers:
    seq_model.add(layer)
    
seq_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [77]:
# 1000 categories, pop off output layer to work only specific categories
seq_model._layers.pop()

seq_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [78]:
# freeze weights to prevent constantly updating weights
for layer in seq_model._layers:
    layer.trainable = False

In [79]:
seq_model.add(Dense(5, activation='softmax')) # 5 categories 

seq_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [87]:
seq_model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train = train_it.n//train_it.batch_size
seq_model.fit_generator(generator=train_it,
                   steps_per_epoch=step_size_train,
                   epochs=7)

Epoch 1/7
6/6 [==============================] - 2s 250ms/step - loss: 1.4456 - accuracy: 0.8085
Epoch 2/7
6/6 [==============================] - 1s 200ms/step - loss: 1.4355 - accuracy: 0.7926
Epoch 3/7
6/6 [==============================] - 1s 213ms/step - loss: 1.4201 - accuracy: 0.8085
Epoch 4/7
6/6 [==============================] - 1s 216ms/step - loss: 1.4045 - accuracy: 0.8490
Epoch 5/7
6/6 [==============================] - 1s 216ms/step - loss: 1.4056 - accuracy: 0.7989
Epoch 6/7
6/6 [==============================] - 1s 206ms/step - loss: 1.3885 - accuracy: 0.8125
Epoch 7/7
6/6 [==============================] - 1s 200ms/step - loss: 1.3778 - accuracy: 0.8245


In [88]:
seq_model.predict(test_batchX)

array([[0.21218693, 0.16658032, 0.18717825, 0.18589532, 0.24815917],
       [0.22785397, 0.1684405 , 0.2020389 , 0.2112219 , 0.19044477],
       [0.21290156, 0.1666764 , 0.18703029, 0.18598758, 0.2474041 ],
       [0.22613029, 0.16851784, 0.1839518 , 0.18904561, 0.23235449],
       [0.22893275, 0.16492856, 0.17033274, 0.25557202, 0.18023388],
       [0.21292599, 0.16726977, 0.18708685, 0.187101  , 0.24561638],
       [0.21186571, 0.16653687, 0.18724473, 0.18585293, 0.24849981],
       [0.2133141 , 0.16756558, 0.18687406, 0.1883527 , 0.24389362],
       [0.21186571, 0.16653687, 0.18724473, 0.18585293, 0.24849981],
       [0.2119089 , 0.16663772, 0.1871709 , 0.18607971, 0.24820282],
       [0.22906333, 0.16478902, 0.17000662, 0.25680918, 0.17933182],
       [0.22355197, 0.16642493, 0.17747885, 0.22800986, 0.20453438],
       [0.22911754, 0.16518164, 0.17259087, 0.2528554 , 0.18025464],
       [0.21186699, 0.16653767, 0.18724422, 0.18585491, 0.24849622],
       [0.21186574, 0.16653694, 0.

In [89]:
prediction = seq_model.predict(test_batchX)

# convert the probabilities to class labels
#label = decode_predictions(pred)

for row in range(len(prediction)):
    biggest = np.argmax(prediction[row])
    for val in range(len(prediction[row])):
        if(val == biggest):
            prediction[row][val] = 1.
        else:
            prediction[row][val] = 0.

# retrieve the most likely result, e.g. highest probability
prediction == test_batchy

array([[False,  True,  True,  True, False],
       [False,  True, False,  True,  True],
       [False,  True,  True,  True, False],
       [False,  True,  True,  True, False],
       [ True, False,  True, False,  True],
       [False,  True,  True,  True, False],
       [ True,  True,  True,  True,  True],
       [ True,  True, False,  True, False],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True, False, False,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True, False, False,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])